# Imports

### Libraries

In [230]:
import os
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tabulate import tabulate

import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torchvision import transforms, models

# Resources Import

In [231]:
!git clone https://github.com/Luca-Olivieri/NAML_project.git

fatal: destination path 'NAML_project' already exists and is not an empty directory.


### Resources paths

In [232]:
local_path = "/content/"
project_path = local_path+"NAML_project"
dataset_path = local_path+"neuroendocrine_"

### Scripts

In [233]:
import sys

sys.path.append(f"{project_path}/src/")

In [234]:
import models_manager
import data_manager
import eval_manager

### Seeds

In [235]:
seed_value = 42

np.random.seed(seed_value)  # NumPy
torch.manual_seed(seed_value)  # PyTorch CPU
torch.cuda.manual_seed_all(seed_value)  # PyTorch GPU

# Model Import

Import one of the pre-trained models.

Available models:

- Inception v3
- AlexNet

In [236]:
model_name = "inceptionv3"
# model_name = "AlexNet"

Select version:

In [237]:
version = "v1"

In [238]:
model = models_manager.import_model(model_name)

In [239]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [240]:
models_manager.setup_model(model, device)

# Data Ingestion

Download dataset from GitHub:

In [241]:
!git clone https://github.com/cialab/neuroendocrine_

fatal: destination path 'neuroendocrine_' already exists and is not an empty directory.


# Training/Loading

Control the training/loading and saving functionalities:

- **training**: if yes, performs the computational training of the model. Otherwise, downloads and loads the already trained parameters.
- **save**: if yes, saves the current state of the model (regardless of the fact that the training has been performed or not).

**Important**: write operations are performed even if files are already present in the runtime. Hence, old files are over-written.

In [242]:
training = True
save = False # regardless of the actual training

Dataset is very unbalanced, so I create a dataset containing 50% tumor and 50% non tumor subimages.
I tried to trained on the original unbaiased dataset and the network pretty much always predicted tumor (because in training 90% of subimages are tumor)

In [243]:
if training == True:

	batch_size = 100
	undersampled_loader = data_manager.create_dataloader(dataset_path, batch_size, removeIndex = 0)

	# optimizer = optim.SGD(inceptionv3.parameters(), lr=0.001, momentum=0.9)
	optimizer = optim.Adam(model.parameters(), lr=0.001)

	# Proceed with the training loop
	models_manager.train(model, optimizer, undersampled_loader, device,
	                     num_epochs=1, max_train=100, print_every=10)

else:

	# Load custom pre-trained parameters
	models_manager.load_params(model, local_path, project_path, version, device)

Shape of the train_images array: (778721, 64, 64, 3)
Shape of the train_labels array: (778721,)
Shape of the val_images array: (39525, 64, 64, 3)
Shape of the val_labels array: (39525,)
Iteration [10/1135], Epoch [1/1], Loss: 0.4985
Iteration [20/1135], Epoch [1/1], Loss: 0.4051
Iteration [30/1135], Epoch [1/1], Loss: 0.3529
Iteration [40/1135], Epoch [1/1], Loss: 0.3224


KeyboardInterrupt: 

### Parameters saving

In [ ]:
if save == True:
   models_manager.save_params(model, local_path)

# Testing

Control the prediction/loading and writing functionalities:

- **pred**: if yes, performs the computational prediction of a selected image's segmentation. Otherwise, loads the file avg.png from the Colab runtime (if it is not present, it downloads it from th GitHub project's repo).
- **avg_write**: if yes, write to the Colab runtime the raw output of the model based on the chosen image (regardless of the fact that the computation has been performed or not).
- **output_write**: if yes, write to the Colab runtime the thresholded output of the model based on the chosen image (regardless of the fact that the computation has been performed or not).

**Important**: write operations are performed even if files are already present in the runtime. Hence, old files are over-written.

In [ ]:
compute = False # If true, compute the mask prediction of a chosen image; otherwise, consider the already pre-computed prediction
avg_write = False # Write the avg photo to Colab, regardless of the actual computation or not
output_write = False # Write the final output photo to Colab

In [ ]:
img_idx = 0 # index of image chosen for evaluation (loaded avg.png refers to -1)

### Target visualisation

Available photos:

In [ ]:
images_paths = data_manager.get_images(dataset_path, test=False)
masks_paths = data_manager.get_images(dataset_path, test=True)

In [ ]:
images_paths

Considered photo:

In [ ]:
images_paths[img_idx], masks_paths[img_idx]

In [ ]:
testImage = cv2.imread(str(images_paths[img_idx]))
testMask = cv2.imread(str(masks_paths[img_idx]))

plt.imshow(testImage)
plt.imshow(testMask, alpha=0.5)

### Mask prediction

In [ ]:
if compute == True:
  avg = models_manager.predict(model, device, testImage)

else:
  avg = models_manager.get_prediction(
      model, local_path, project_path, version)

### Mask predition saving

In [ ]:
if avg_write == True:
   cv2.imwrite('avg.png', avg*255)

Visualisations

Raw output of the segmentation:

In [ ]:
raw_output = models_manager.beautify_output(avg)
models_manager.visualise_output(raw_output)

Thresholded output:

In [ ]:
thr = 0.65 # threshold

thr_output = models_manager.threshold_output(avg, thr)
models_manager.visualise_output(thr_output)

Prediction saving:

In [ ]:
models_manager.save_prediction(thr_output)

# Evaluation

###Metrics

In [ ]:
tumor = True

In [ ]:
y_true = eval_manager.get_true_labels(testMask, tumor)
y_pred = eval_manager.get_pred_labels(thr_output, tumor)

Compute metrics:
- Accuracy
- ...

In [ ]:
acc = eval_manager.pixel_accuracy(y_true, y_pred)
prec = eval_manager.pixel_precision(y_true, y_pred)
recall = eval_manager.pixel_recall(y_true, y_pred)
iou = eval_manager.intersection_over_union(y_true, y_pred)
dice = eval_manager.dice_coefficient(y_true, y_pred)

Show metric table:

In [ ]:
eval_manager.show_metrics(acc, prec, recall, iou, dice)

### Confusion matrix:

In [ ]:
conf_matrix = eval_manager.conf_matrix(y_true, y_pred)
conf_norm_matrix = eval_manager.conf_matrix(y_true, y_pred,
                                            normalise="true")

In [ ]:
eval_manager.show_conf_matrices(conf_matrix, conf_norm_matrix)

### Visualisation

Visualisation of prediction border onto input image:

In [ ]:
eval_manager.visualize_bordered_mask(testImage, thr_output, radius=2)